# imports

In [23]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

## read data

In [2]:
PATH='storage/chest_xray/'
!ls {PATH}
sz = 224
batch_size=64

models	test  tmp  train  val


In [3]:
train_data_dir = f'{PATH}train'
# learn to rename in jupyter notebook
validation_data_dir = f'{PATH}test' 

## first try with default values

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(sz, sz),
                                                   batch_size=batch_size,
                                                   class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        shuffle=False,
                                                       target_size=(sz, sz),
                                                       batch_size=batch_size,
                                                       class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

/opt/conda/envs/fastai/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [6]:
model = Model(inputs=base_model.input, outputs=predictions)
# freeze all the layers
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size,
                    epochs=3, workers=4,
                   validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
81/81 [==============================] - 48s 597ms/step - loss: 4.0232 - acc: 0.7405 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 2/3
81/81 [==============================] - 41s 500ms/step - loss: 4.1239 - acc: 0.7413 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 3/3
81/81 [==============================] - 41s 504ms/step - loss: 4.1207 - acc: 0.7415 - val_loss: 5.9784 - val_acc: 0.6250
CPU times: user 6min 4s, sys: 36.6 s, total: 6min 40s
Wall time: 2min 10s


 unfreeze some layers to have some fine tune o

In [8]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit_generator(train_generator, train_generator.n // batch_size,
                   epochs=3, workers=4,
                   validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
81/81 [==============================] - 57s 699ms/step - loss: 4.1268 - acc: 0.7411 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 2/3
81/81 [==============================] - 50s 623ms/step - loss: 4.0809 - acc: 0.7440 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 3/3
81/81 [==============================] - 50s 621ms/step - loss: 4.0963 - acc: 0.7431 - val_loss: 5.9784 - val_acc: 0.6250


# start from small image size with unfreeze all the layers 

## first try with size = 64 

In [11]:
sz = 64
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(sz, sz),
                                                   batch_size=batch_size,
                                                   class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        shuffle=False,
                                                       target_size=(sz, sz),
                                                       batch_size=batch_size,
                                                       class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [12]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size,
                    epochs=3, workers=4,
                   validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
81/81 [==============================] - 19s 234ms/step - loss: 0.6330 - acc: 0.8997 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 2/3
81/81 [==============================] - 17s 215ms/step - loss: 0.1585 - acc: 0.9452 - val_loss: 5.9592 - val_acc: 0.6250
Epoch 3/3
81/81 [==============================] - 17s 212ms/step - loss: 0.1521 - acc: 0.9439 - val_loss: 5.9715 - val_acc: 0.6250
CPU times: user 3min 24s, sys: 11.9 s, total: 3min 36s
Wall time: 53.5 s


 unfreeze all the layers

In [13]:
for layer in model.layers[:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit_generator(train_generator, train_generator.n // batch_size,
                   epochs=3, workers=4,
                   validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
81/81 [==============================] - 31s 377ms/step - loss: 0.2943 - acc: 0.9165 - val_loss: 4.5449 - val_acc: 0.6170
Epoch 2/3
81/81 [==============================] - 19s 235ms/step - loss: 0.1676 - acc: 0.9437 - val_loss: 3.9095 - val_acc: 0.6843
Epoch 3/3
81/81 [==============================] - 19s 234ms/step - loss: 0.1482 - acc: 0.9466 - val_loss: 3.3029 - val_acc: 0.6923


## size = 128 , unfreeze all the layers

In [15]:
sz = 128
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(sz, sz),
                                                   batch_size=batch_size,
                                                   class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        shuffle=False,
                                                       target_size=(sz, sz),
                                                       batch_size=batch_size,
                                                       class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [16]:
for layer in model.layers[:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.fit_generator(train_generator, train_generator.n // batch_size,
                   epochs=3, workers=4,
                   validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
81/81 [==============================] - 58s 717ms/step - loss: 0.1942 - acc: 0.9298 - val_loss: 6.4588 - val_acc: 0.5657
Epoch 2/3
81/81 [==============================] - 43s 530ms/step - loss: 0.1284 - acc: 0.9545 - val_loss: 1.9586 - val_acc: 0.7917
Epoch 3/3
81/81 [==============================] - 43s 530ms/step - loss: 0.1078 - acc: 0.9632 - val_loss: 2.1326 - val_acc: 0.7692


In [29]:
model.save('keras_128_64')

In [31]:
from keras.models import load_model